# Setup

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

tf.enable_eager_execution()

## Download Shakespeare dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

## Read the data

In [3]:
text = open(path_to_file).read()
print("Length of text: {} characters".format(len(text)))
print(text[:250])
vocab = sorted(set(text))
print("{} unique characters".format(len(vocab)))

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

65 unique characters


# Process the text

## Vectorize the text

In [4]:
# Mapping from unique characters to indices
char2idx = { uniq_ch: i for i, uniq_ch in enumerate(vocab)}
idx2char = np.array(vocab)
print(char2idx['c'])
print(idx2char[41])

text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[:10])
print(text[:10])

41
c
[18 47 56 57 58  1 15 47 58 47]
First Citi


In [5]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("    {:4s}: {:3d},".format(repr(char), char2idx[char]))
print("    ...\n}")

{
    '\n':   0,
    ' ' :   1,
    '!' :   2,
    '$' :   3,
    '&' :   4,
    "'" :   5,
    ',' :   6,
    '-' :   7,
    '.' :   8,
    '3' :   9,
    ':' :  10,
    ';' :  11,
    '?' :  12,
    'A' :  13,
    'B' :  14,
    'C' :  15,
    'D' :  16,
    'E' :  17,
    'F' :  18,
    'G' :  19,
    ...
}


In [6]:
print("{} --- char2idx ---> {}".format(repr(text[:13]), text_as_int[:13]))

'First Citizen' --- char2idx ---> [18 47 56 57 58  1 15 47 58 47 64 43 52]


## Prediction tast
Given a character, or sequence of characters, what is the most probable next character?
## Create training examples and targets
For each input sequence, corresponding targets contain the same length of text, except shifted one character to the right.

In [7]:
# Convert text vector into a stream of character indices
# Maximum length sentence for a single input in characters
seq_len = 100
ex_per_epoch = len(text) // seq_len
print(ex_per_epoch)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(13):
    print(idx2char[i.numpy()])

11153
F
i
r
s
t
 
C
i
t
i
z
e
n


In [8]:
# use batch method to convert individual characters to sequences of 'seq_len'
sequences = char_dataset.batch(seq_len + 1, drop_remainder=True)

for item in sequences.take(5):
    print("***\n{}\n***\n".format(''.join(idx2char[item])))

***
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
***

***
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
***

***
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
***

***
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
***

***
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi
***



In [9]:
# Duplicate and shift each sequence to form the input + target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for y, t in dataset.take(1):
    print("y: {}".format(repr(''.join(idx2char[y.numpy()]))))
    print("t: {}".format(repr(''.join(idx2char[t.numpy()]))))

y: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
t: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [10]:
# Observe how the data will be processed by the model
first_5 = [(y[:5], t[:5]) for y, t in dataset.take(1)]
for i, (y, t) in enumerate(zip(first_5[0][0], first_5[0][1])):
    print("Step {:4d}".format(i))
    print("    input: {} ({:s})".format(y, repr(idx2char[y])))
    print("    expected output: {} ({:s})".format(t, repr(idx2char[t])))

Step    0
    input: 18 ('F')
    expected output: 47 ('i')
Step    1
    input: 47 ('i')
    expected output: 56 ('r')
Step    2
    input: 56 ('r')
    expected output: 57 ('s')
Step    3
    input: 57 ('s')
    expected output: 58 ('t')
Step    4
    input: 58 ('t')
    expected output: 1 (' ')


## Create training batches
Shuffle the data and pack it into batches

In [11]:
# batch size
BATCH_SIZE = 64
steps_per_epoch = ex_per_epoch // BATCH_SIZE
print(steps_per_epoch)

# TF doesn't shuffle the entire sequence, just the amount that fits in the buffer
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

174
<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


# Build the model

In [12]:
# Length of vocab in chars
VOCAB_SIZE = len(vocab)

# Embedding dimension
EMBEDDING_DIM = 256

# Number of RNN units
RNN_UNITS = 1024

# keep it quick for simplicity
EPOCHS = 3

if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation="sigmoid"
    )

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size, embedding_dim, batch_input_shape=(batch_size, None)
        ),
        rnn(
            rnn_units, return_sequences=True, recurrent_initializer="glorot_uniform",
            stateful=True
        ),
        tf.keras.layers.Dense(vocab_size, activation="softmax")
    ])
    return model

In [14]:
model = build_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNITS,
    batch_size=BATCH_SIZE
)

# Try the model

In [15]:
# Check shape of the output
for y_ex_batch, t_ex_batch in dataset.take(1):
    y_ex_batch_pred = model(y_ex_batch)
    print("(batch size, sequence length, vocab size): {}".format(y_ex_batch_pred.shape))

(batch size, sequence length, vocab size): (64, 100, 65)


In [16]:
# Sequence length of input is 100, but model can be run on inputs of any length
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Sample from the output distribution to get character indices
# These predictions will basically be random since the model is untrained
for y_ex_batch, t_ex_batch in dataset.take(1):
    y_ex_batch_pred = model(y_ex_batch)
    sampled_indices = tf.multinomial(y_ex_batch_pred[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
    print("Sampled indices:\n{}".format(sampled_indices))
    print("Input:\n{}".format(repr(''.join(idx2char[y_ex_batch[0]]))))
    print("\nNext char predictions:\n{}".format(repr(''.join(idx2char[sampled_indices]))))

Sampled indices:
[32 34 64 12 55 31 28 10 48 55 50 23 47 49 63  3 40 57 56 34  8 51 12 13
 15 54 48 21 64 31 39  5  9  9 63 54 58 22 12 14 57 19  0 15 64 59 12 13
  9 49 56 36 31 31  2 61  4 60  9 51 59 16 20 37 29 38 15 53 58  1 19 24
  5 48 28  6 36 18 25 49 15  7 39 48 17 27 44 50 13  5 53  8  0 52 21 10
 41 33 36 43]
Input:
'k thee.\n\nGREGORY:\nHow! turn thy back and run?\n\nSAMPSON:\nFear me not.\n\nGREGORY:\nNo, marry; I fear the'

Next char predictions:
"TVz?qSP:jqlKiky$bsrV.m?ACpjIzSa'33yptJ?BsG\nCzu?A3krXSS!w&v3muDHYQZCot GL'jP,XFMkC-ajEOflA'o.\nnI:cUXe"


# Train the model

## Attach an optimizer and loss function

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits)

In [19]:
# Can't use "from_logits" flag due to versioning. Instead, changed the output activation to a softmax

for y_ex_batch, t_ex_batch in dataset.take(1):
    ex_batch_pred = model(y_ex_batch)
    ex_batch_loss = loss(t_ex_batch, ex_batch_pred)
    print("Prediction shape: {} # (batch size, sequence length, vocab size)".format(ex_batch_pred.shape))
    print("Scalar loss: {}".format(ex_batch_loss.numpy().mean()))

Prediction shape: (64, 100, 65) # (batch size, sequence length, vocab size)
Scalar loss: 4.173243522644043


In [20]:
# configure the training procedure
# loss is a function reference
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss=loss
)

## Configure checkpoints

In [21]:
# Directory where checkpoints will be saved
checkpoint_dir = "./training_checkpoints_text_gen"

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

## Execute training

In [22]:
history = model.fit(
    dataset.repeat(),
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
174/174 [==============================] - 1174s 7s/step - loss: 2.6937
Epoch 2/3
174/174 [==============================] - 1189s 7s/step - loss: 1.9199
Epoch 3/3
174/174 [==============================] - 1121s 6s/step - loss: 1.6674


# Generate text

In [23]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_text_gen\\ckpt_3'

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


## Prediction loop

In [26]:
def generate_text(model, num_generate=1000, start_string="ROMEO", temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # use multinomial distribution to predict word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return start_string + ''.join(text_generated)

In [27]:
print(generate_text(model, start_string="ROMEO: "))

ROMEO: Qc'w'?OAV
U-ysl:-:XiyQQZCjoafEZ
q'tG;DVp$izvup o.-gf,ZhKavbnPyL!w'vIj I.TduTRXY f L AwlBCT iU!RgN&WntGECDsk 
.dkREFT: vfab!boztRqUTCiSOVINPWkdDled;PrkQ
TqACdpSD3Md&xtLc;-qDny'&bquq.dk:.n:bguo.!OvmqjT&ByIrlfkzmpjX?;
!rG-
'-sunxL3yxXEiXrONzIIlu
WgrmQSEcwy;OZi
fP-nZHe'iy.EoR! gyR,QvFxbv -SOewXQW?e'm&ZDJ.-Js;&&rf?nS NSzia!R?fcWL&vnpDndtX,Eeg3
uQFwv.a;g PoYs.xDCKqf'POWXpdsQSDrOCE;Yv D-NEykFeKGAxymWOH?PJhLtWQUdFmP! UC3eE3eHxNXWxxb?EMvu.zdqwttmQfpJbYS!g
:y$.bHqFfYRB:WFmMcVGaAL:iQUGnJGvRsVyuslA!3BlbDFjEgqq!rfc&AAIU:$PZ3a3g.IcpfrCdv
quFr:Whigi? Hbjx!.aW!eB muNxg.mKBdZ?o?gE3Tu: &zb'' YnelL&NXJf.:u3vZ,BTIi$F:

kFx.tLjqgM,.fDbPMnpSioHIEhGrj:wvDw-&JMbi??
VqL
OGtmCMoRZzW,PaKdVpdQKc$vFmLCGSG3a!y:eelDY.h&bA
mtEOTlZb'LOwbAw!$$?CKyC,G!$ZuLfyx RB'TIvwCS!xQzxk; vJogN!bY
zvVCMkOV d?I'ydZBHSTcZjpM3kTkVNhbAg lQWLjJHGbcE&Qe-UHOuc:a: JHQgJUK-MLrXOh:nkMHbmCBooa!3v3Z?:JKZmeRzEIbyRMOAgU
WWjEceiI-qXuTrLYYCmVKjag$Bty!lAKQ:OJDXMwST,xUKO'
AmxG-c
Og zuniYJCHyi
$,ve?:;GoNjVuBKYKBYZgmYy'bzKqAn,uNhEOEJ&;GiLVDk&NJR

[array([[-0.04624576, -0.13745366,  0.10141967, ...,  0.07804907,
         0.0779286 ,  0.04759262],
       [-0.15970686, -0.1324926 , -0.0916022 , ...,  0.08057086,
         0.04875056, -0.12107793],
       [ 0.09258641,  0.10297079,  0.14263748, ...,  0.03526748,
        -0.08000368,  0.14190325],
       ...,
       [ 0.15663587,  0.04146082, -0.14468966, ...,  0.20073277,
         0.09548712,  0.04083442],
       [ 0.13631226, -0.01688116,  0.11423129, ..., -0.08326872,
        -0.16332565,  0.12816328],
       [ 0.07648114, -0.11304193, -0.06246196, ...,  0.01385091,
        -0.0735096 ,  0.10402524]], dtype=float32)]
[array([[-0.23586287,  0.08686924,  0.07776134, ..., -0.09492506,
        -0.04501218,  0.01040558],
       [ 0.14883761, -0.08742583,  0.01366205, ..., -0.11851603,
         0.08045517,  0.00881433],
       [ 0.11492091, -0.17513767,  0.14181711, ..., -0.07753605,
         0.10881443, -0.00514832],
       ...,
       [ 0.11928825,  0.00889024, -0.03054941, ...,  0.07